In [8]:
def sys_path_append(path):
    import sys
    if path not in sys.path:
        sys.path.append(path)

sys_path_append('../source')
sys_path_append('../utilities')

from common_imports import set_num_threads
set_num_threads(2) # Set the number of cores. Must be executed before importing numpy&numba

import rbm
import RNA_utils
import numpy

In [9]:
all_data, all_labels = RNA_utils.load_RNA_MSA('../data/RF00162_trimmed.fasta', with_labels = True)
# discard sequences with strange letters
all_labels = all_labels[numpy.invert((all_data == 5).any(axis=1))]
all_data = all_data[numpy.invert((all_data == 5).any(axis=1))]

In [14]:
def train_test_split(data, frac=0.8, shuffle=True):
    i = round(frac * len(data))
    if shuffle:
        p = numpy.random.permutation(len(data))
        data_ = data[p]
    else:
        data_ = data
    train_data, tests_data = data_[:i], data_[i:]
    return train_data, tests_data

In [15]:
train_data, tests_data = train_test_split(all_data)

In [16]:
RBM = rbm.RBM(n_v=108, n_cv=5, n_h=100, visible='Potts', hidden='dReLU')
record = RBM.fit(train_data, n_iter=400, batch_size=256, N_MC=5, verbose=1, l2_fields=1e-3, l1b=0.01, 
                 record=['PL', 'PL_test'], record_interval=1, data_test=tests_data)

Iteration number 0, pseudo-likelihood: -0.81
Starting epoch 1
[RBM] Iteration 1, time = 7.05s, pseudo-likelihood = -0.62
Starting epoch 2
[RBM] Iteration 2, time = 7.49s, pseudo-likelihood = -0.57
Starting epoch 3
[RBM] Iteration 3, time = 6.96s, pseudo-likelihood = -0.53
Starting epoch 4
[RBM] Iteration 4, time = 7.51s, pseudo-likelihood = -0.51
Starting epoch 5
[RBM] Iteration 5, time = 6.32s, pseudo-likelihood = -0.50
Starting epoch 6


In [ ]:
# Since training takes a long time, we can just load a pre-trained model
with open('../data/RBM_nh_100_l1b_0.01.pickle', 'rb') as file:
    RBMdict = pickle.load(file)
RBM = RBMdict['RBM']
record = RBMdict['record']